In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

col_names = []
with open('../data/Faults27x7_var','r') as f:
    for line in f:
        col_names.append(line.strip())
        
train_org = pd.read_csv('../data/train.csv')
test_org = pd.read_csv('../data/test.csv')
org_data = pd.read_csv('../data/Faults.NNA', delimiter='\s', engine='python', names=col_names)

# logistic regression
X = train_org.drop(['id','Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps','Other_Faults'], axis=1)
pastry = train_org['Pastry'].copy()
z_scratch = train_org['Z_Scratch'].copy()
k_scatch = train_org['K_Scatch'].copy()
stains = train_org['Stains'].copy()
dirtiness = train_org['Dirtiness'].copy()
bumps = train_org['Bumps'].copy()
other_faults = train_org['Other_Faults'].copy()

ys = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
y_names = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

class KMeansTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=0)
        
    def fit(self,X, y=None):
        self.kmeans.fit(X)
        return self
        
    def transform(self,X):
        labels = self.kmeans.predict(X)
        return np.c_[X, labels]

In [2]:
y_params_dict = {}
for y_nm in y_names:
    y_params_dict[y_nm] = None

In [3]:
y_scores = []

In [4]:
for y,y_name in zip(ys,y_names):
    
    print(f'trial: {y_name}\n')
    
    def objective_lgb(trial):
    
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        params = dict(
            n_estimators = trial.suggest_int('n_estimators',100,500),
            max_depth = trial.suggest_int('max_depth',2,64),
            num_leaves = trial.suggest_int('num_leaves',2,128),
            learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
            min_child_samples = trial.suggest_int('min_child_samples',2,500),
            min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
            subsample = trial.suggest_float('subsample', 0.33,0.85),
            colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
            reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
        )
        
        lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
        
        pipe = Pipeline(
            steps = [
                ('scaler', StandardScaler()),
                ('model', lgbc)
            ]
        )
        
        score = np.mean(cross_val_score(pipe, X,pastry, scoring='roc_auc', cv= cvo))
        return score
    
    study_lgb = optuna.create_study(direction='maximize')
    
    study_lgb.optimize(objective_lgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)
    
    best_params = study_lgb.best_params
    y_params_dict[y_name] = best_params
    
    best_score = study_lgb.best_value
    y_scores.append((y_name,best_score))
    
    time.sleep(5)
    clear_output()

In [5]:
y_params_dict

{'pastry': {'n_estimators': 414,
  'max_depth': 5,
  'num_leaves': 51,
  'learning_rate': 0.027772778201643363,
  'min_child_samples': 163,
  'min_child_weight': 6.83071327201467,
  'subsample': 0.49696742935005617,
  'colsample_bylevel': 0.35120262401687735,
  'reg_alpha': 0.09765235819012545,
  'reg_lambda': 0.05441956449748974},
 'z_scratch': {'n_estimators': 247,
  'max_depth': 13,
  'num_leaves': 26,
  'learning_rate': 0.023970308041410808,
  'min_child_samples': 229,
  'min_child_weight': 7.367478870340577,
  'subsample': 0.39095982111246597,
  'colsample_bylevel': 0.34924153428375315,
  'reg_alpha': 0.05387841564588599,
  'reg_lambda': 0.06614746132682563},
 'k_scatch': {'n_estimators': 404,
  'max_depth': 11,
  'num_leaves': 14,
  'learning_rate': 0.01981017812667226,
  'min_child_samples': 306,
  'min_child_weight': 1.0908593258138115,
  'subsample': 0.5677335874920294,
  'colsample_bylevel': 0.34897570166413927,
  'reg_alpha': 0.08436824936832184,
  'reg_lambda': 0.0919585055

In [6]:
y_params_dict['z_scratch']

{'n_estimators': 247,
 'max_depth': 13,
 'num_leaves': 26,
 'learning_rate': 0.023970308041410808,
 'min_child_samples': 229,
 'min_child_weight': 7.367478870340577,
 'subsample': 0.39095982111246597,
 'colsample_bylevel': 0.34924153428375315,
 'reg_alpha': 0.05387841564588599,
 'reg_lambda': 0.06614746132682563}

In [7]:
pastry_params = y_params_dict['pastry']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [8]:
z_scratch_params = y_params_dict['z_scratch']

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [9]:
k_scatch_params = y_params_dict['k_scatch']

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [10]:
stains_params = y_params_dict['stains']

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [11]:
dirtiness_params = y_params_dict['dirtiness']

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [12]:
bumps_params = y_params_dict['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [13]:
other_faults_params = y_params_dict['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [15]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [16]:
sub.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.556846,0.001224,0.003118,0.000023,0.016610,0.150386,0.309808
1,19220,0.277344,0.013506,0.004721,0.000044,0.148470,0.137887,0.315844
2,19221,0.001745,0.020020,0.052210,0.000121,0.001968,0.317900,0.472201
3,19222,0.125359,0.001467,0.000452,0.000926,0.005287,0.336195,0.433519
4,19223,0.002675,0.001682,0.000493,0.001442,0.006612,0.631552,0.375618


In [17]:
ts = 0
for yn,score in y_scores:
    ts += score
    
print(ts/7)

0.8715945082358949


In [18]:
sub.to_csv('../submissions/m5.csv', index=False)

In [19]:
for yn,score in y_scores:
    print(score)

0.8715724463430068
0.8716229666454796
0.8711610211743859
0.8718911456624303
0.8716650442140532
0.871617374898657
0.8716315587132517


In [20]:
#1
pastry_params = {'n_estimators': 280,
 'max_depth': 9,
 'num_leaves': 18,
 'learning_rate': 0.02148602951813924,
 'min_child_samples': 341,
 'min_child_weight': 6.361004402846124,
 'subsample': 0.49585801139053814,
 'colsample_bylevel': 0.35930633871573076,
 'reg_alpha': 0.06447122051496705,
 'reg_lambda': 0.016497887759421646}

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [21]:
z_scratch_params = {'n_estimators': 296,
 'max_depth': 3,
 'num_leaves': 74,
 'learning_rate': 0.03879534308586422,
 'min_child_samples': 298,
 'min_child_weight': 2.4215929637347218,
 'subsample': 0.47753292687280846,
 'colsample_bylevel': 0.35906419523398436,
 'reg_alpha': 0.057670762264904175,
 'reg_lambda': 0.0427161308925542}

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [22]:
k_scatch_params = {'n_estimators': 271,
 'max_depth': 29,
 'num_leaves': 118,
 'learning_rate': 0.02560845874325205,
 'min_child_samples': 449,
 'min_child_weight': 1.802873155395419,
 'subsample': 0.7902749625886126,
 'colsample_bylevel': 0.3433292920431201,
 'reg_alpha': 0.047379430530416024,
 'reg_lambda': 0.010007517965380167}

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [23]:
stains_params = {'n_estimators': 266,
 'max_depth': 5,
 'num_leaves': 75,
 'learning_rate': 0.027151102431374634,
 'min_child_samples': 170,
 'min_child_weight': 1.024782187730875,
 'subsample': 0.5803990695479041,
 'colsample_bylevel': 0.6394813204481101,
 'reg_alpha': 0.09477440109735585,
 'reg_lambda': 0.05246070547701958}

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [24]:
dirtiness_params = {'n_estimators': 100,
 'max_depth': 13,
 'num_leaves': 62,
 'learning_rate': 0.04095550709994935,
 'min_child_samples': 331,
 'min_child_weight': 0.1953843881250878,
 'subsample': 0.4674944721594767,
 'colsample_bylevel': 0.33072889947558426,
 'reg_alpha': 0.07067954909592045,
 'reg_lambda': 0.0017098736746246962}

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [25]:
bumps_params = y_params_dict['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [26]:
other_faults_params = y_params_dict['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [27]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [28]:
sub.to_csv('../submissions/m5_1.csv', index=False)

In [29]:
(0.8721124548680992 + 0.9613490481775984 + 0.9860986294660549 + 0.9931795666268013 + 0.8967476825823397 + 0.871617374898657 + 0.8716315587132517)/7

0.9218194736189717